# Building up the pipeline to sample street views

First, I want to see how Google APIs work and how i can combine them together. 

Specifically I want to check out:

- Places API 
- StreetView API
 

I already got an API key and set it in my env file.

In [32]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
# Global API key configuration
import os
API_KEY_GMP = os.getenv("GOOGLE_API_KEY")
assert API_KEY_GMP, "Missing GOOGLE_API_KEY in environment."


Call the places API in a radius around Bologna:

In [ ]:
import os
import requests
import json

# Example request to Places API - Text Search restricted to Bologna
query = "Giardini Margherita"
url = "https://places.googleapis.com/v1/places:searchText"

headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": API_KEY_GMP,
    'X-Goog-FieldMask' : '*'   # many fields available, use '*' to get all
}

#useful: location contains lat and lon

# Bologna coordinates
bologna_lat = 44.4949
bologna_lng = 11.3426
radius_meters = 5000  # 5km radius

payload = {
    "textQuery": query,
    "locationBias": {
        "circle": {
            "center": {
                "latitude": bologna_lat,
                "longitude": bologna_lng
            },
            "radius": radius_meters
        }
    },
    "maxResultCount": 5  # Limit to 5 results
}

response = requests.post(url, headers=headers, json=payload)
data = response.json()

print("Status Code:", response.status_code)
print(f"\nSearching for '{query}' in Bologna (radius: {radius_meters}m)")
print(f"\nNumber of results: {len(data.get('places', []))}")
print("\nResponse:")
print(json.dumps(data, indent=2))


Status Code: 200

Searching for 'Giardini Margherita' in Bologna (radius: 5000m)

Number of results: 1

Response:
{
  "places": [
    {
      "name": "places/ChIJLwKOhMXUf0cRiLrCD6oTfS0",
      "id": "ChIJLwKOhMXUf0cRiLrCD6oTfS0",
      "types": [
        "hiking_area",
        "tourist_attraction",
        "park",
        "sports_activity_location",
        "point_of_interest",
        "establishment"
      ],
      "nationalPhoneNumber": "051 203040",
      "internationalPhoneNumber": "+39 051 203040",
      "formattedAddress": "Viale Giovanni Gozzadini, 40136 Bologna BO, Italy",
      "addressComponents": [
        {
          "longText": "Viale Giovanni Gozzadini",
          "shortText": "Viale Giovanni Gozzadini",
          "types": [
            "route"
          ],
          "languageCode": "it"
        },
        {
          "longText": "Bologna",
          "shortText": "Bologna",
          "types": [
            "locality",
            "political"
          ],
          "langu

Return only needed fields: for example location, which contains lat & lon

In [ ]:
import os
import requests
import json

# Example request to Places API - Text Search restricted to Bologna
query = "Giardini Margherita"
url = "https://places.googleapis.com/v1/places:searchText"

headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": API_KEY_GMP,
    'X-Goog-FieldMask' : 'places'   # many fields available, use '*' to get all
}

#useful: location contains lat and lon

# Bologna coordinates
bologna_lat = 44.4949
bologna_lng = 11.3426
radius_meters = 5000  # 5km radius

payload = {
    "textQuery": query,
    "locationBias": {
        "circle": {
            "center": {
                "latitude": bologna_lat,
                "longitude": bologna_lng
            },
            "radius": radius_meters
        }
    },
    "maxResultCount": 5  # Limit to 5 results
}

response = requests.post(url, headers=headers, json=payload)
data = response.json()

print("Status Code:", response.status_code)
print(f"\nSearching for '{query}' in Bologna (radius: {radius_meters}m)")
print(f"\nNumber of results: {len(data.get('places', []))}")
print("\nResponse:")
print(json.dumps(data, indent=2))
print("="*60)
print(data.get('places', [])[0].get('location', {}))


Status Code: 200

Searching for 'Giardini Margherita' in Bologna (radius: 5000m)

Number of results: 1

Response:
{
  "places": [
    {
      "name": "places/ChIJLwKOhMXUf0cRiLrCD6oTfS0",
      "id": "ChIJLwKOhMXUf0cRiLrCD6oTfS0",
      "types": [
        "hiking_area",
        "tourist_attraction",
        "sports_activity_location",
        "park",
        "point_of_interest",
        "establishment"
      ],
      "nationalPhoneNumber": "051 203040",
      "internationalPhoneNumber": "+39 051 203040",
      "formattedAddress": "Viale Giovanni Gozzadini, 40136 Bologna BO, Italy",
      "addressComponents": [
        {
          "longText": "Viale Giovanni Gozzadini",
          "shortText": "Viale Giovanni Gozzadini",
          "types": [
            "route"
          ],
          "languageCode": "it"
        },
        {
          "longText": "Bologna",
          "shortText": "Bologna",
          "types": [
            "locality",
            "political"
          ],
          "langu

In [ ]:
import os
import requests

# Street View Static API URL
url = "https://maps.googleapis.com/maps/api/streetview"

# Use Piazza Maggiore coordinates (city center - has Street View coverage)
lat = 44.4949
lng = 11.3426

params = {
    "location": f"{lat},{lng}",  # Required: lat,lng coordinates
    "size": "640x640",            # Required: image size (max 640x640)
    "heading": 0,                 # Optional: camera direction in degrees (0-360)
    "pitch": 0,                   # Optional: vertical angle (-90 to 90, 0 = level)
    "fov": 90,                    # Optional: zoom level (10-120, higher = zoomed in)
    "key": API_KEY_GMP
}

response = requests.get(url, params=params)

print("Status Code:", response.status_code)
if response.status_code == 200:
    # Save the image
    with open("street_view_test.jpg", "wb") as f:
        f.write(response.content)
    print("Image saved as 'street_view_test.jpg'")
else:
    print(f"Error: {response.text}")

Status Code: 200
Image saved as 'street_view_test.jpg'


## Build the pipeline 

We will write a function that samples `N` points around the radius `R` starting from a fixed location `center`.

It will need to retry because we may encounter locations that do not have street views associated; we will implement this later on.

In [36]:
import random
import math

def sample_points_in_radius(center : tuple[float, float], radius : float, N : int) -> list[tuple[float, float]]:
    """
    Sample N points in a circle of radius R centered at center.

    Args:
        center: tuple of float, the center of the circle (lat, lon)
        radius: float, the radius of the circle in meters
        N: int, the number of points to sample

    Returns:
        list of tuples of float, the sampled points (lat, lon)
    """
    center_lat, center_lon = center
    points = []
    
    for _ in range(N):
        # Generate random angle and distance
        angle = random.uniform(0, 2 * math.pi)
        # Use square root for uniform distribution in circle
        distance = math.sqrt(random.uniform(0, 1)) * radius
        
        # Convert meters to approximate lat/lon offset
        # Rough approximation: 1 degree lat ≈ 111,000m, 1 degree lon ≈ 111,000m * cos(lat)
        lat_offset = distance / 111000
        lon_offset = distance / (111000 * math.cos(math.radians(center_lat)))
        
        # Calculate new coordinates
        new_lat = center_lat + lat_offset * math.cos(angle)
        new_lon = center_lon + lon_offset * math.sin(angle)
        
        points.append((new_lat, new_lon))
    
    return points
    

In [37]:
# Test the function
bologna_center = (44.4949, 11.3426)  # Piazza Maggiore
radius_meters = 1000  # 1km radius
n_points = 5

sampled_points = sample_points_in_radius(bologna_center, radius_meters, n_points)

print(f"Center: {bologna_center}")
print(f"Radius: {radius_meters}m")
print(f"Sampled {len(sampled_points)} points:")
for i, (lat, lon) in enumerate(sampled_points):
    print(f"  {i+1}: ({lat:.6f}, {lon:.6f})")


Center: (44.4949, 11.3426)
Radius: 1000m
Sampled 5 points:
  1: (44.490777, 11.334075)
  2: (44.498317, 11.346206)
  3: (44.502797, 11.347025)
  4: (44.495270, 11.349881)
  5: (44.487481, 11.337039)


In [ ]:
# Street View coverage validation
import os
import requests

API_KEY_GMP = os.getenv("GOOGLE_API_KEY")

# Funzione di Validazione con Street View Metadata API (Richiede l'API_KEY_GMP globale)
def validate_streetview_coverage(lat: float, lng: float):
    """
    Usa Street View Metadata per verificare se esiste un panorama per la location.
    Restituisce (has_coverage: bool, date: Optional[str], pano_id: Optional[str]).
    """
    global API_KEY_GMP

    metadata_url = (
        "https://maps.googleapis.com/maps/api/streetview/metadata?"
        f"location={lat},{lng}&"
        f"key={API_KEY_GMP}"
    )

    try:
        response = requests.get(metadata_url, timeout=5)
        response.raise_for_status()
        data = response.json()

        if data.get('status') == 'OK':
            date = data.get('date', None)
            pano_id = data.get('pano_id', None)
            return True, date, pano_id
        else:
            return False, None, None

    except requests.RequestException:
        return False, None, None


In [ ]:
# Sample only points with Street View coverage (with retries)
import time
from typing import List, Tuple


def sample_points_with_coverage(center: tuple[float, float], radius_m: float, n_points: int,
                                max_attempts: int = 200, request_pause_s: float = 0.0) -> list[tuple[float, float]]:
    """
    Sample points uniformly within radius and keep only those with Street View coverage.
    Stops when n_points are found or max_attempts reached.
    """
    valid_points: List[Tuple[float, float]] = []
    attempts = 0

    while len(valid_points) < n_points and attempts < max_attempts:
        lat, lon = sample_points_in_radius(center, radius_m, 1)[0]
        has_cov, _, _ = validate_streetview_coverage(lat, lon)
        if has_cov:
            valid_points.append((lat, lon))
        attempts += 1
        if request_pause_s:
            time.sleep(request_pause_s)

    return valid_points



In [ ]:
# Use the coverage-aware sampler
bologna_center = (44.4949, 11.3426)
radius_meters = 1000
n_points = 5

max_attempts = 100  # stop if we can't find N within this many attempts
sampled_points = sample_points_with_coverage(bologna_center, radius_meters, n_points,
                                            max_attempts=max_attempts, request_pause_s=0.0)

print(f"Requested {n_points} points; collected {len(sampled_points)} with Street View coverage.")
for i, (lat, lon) in enumerate(sampled_points):
    print(f"  {i+1}: ({lat:.6f}, {lon:.6f})")


In [38]:
# Visualize with folium
import folium

# Create map centered on Bologna
m = folium.Map(location=bologna_center, zoom_start=13)

# Add the center point with a different marker
folium.Marker(
    bologna_center,
    popup='Center (Piazza Maggiore)',
    tooltip='Center',
    icon=folium.Icon(color='red', icon='flag')
).add_to(m)

# Add the sampled points
for i, (lat, lon) in enumerate(sampled_points):
    folium.Marker(
        [lat, lon],
        popup=f'Point {i+1}',
        tooltip=f'Point {i+1}',
        icon=folium.Icon(color='blue', icon='circle')
    ).add_to(m)

# Add a circle to show the radius
folium.Circle(
    location=bologna_center,
    radius=radius_meters,
    color='red',
    fill=False,
    weight=2,
    opacity=0.5
).add_to(m)

# Display the map
m


In [ ]:
# Build Street View URL entries and save to CSV (URLs without API key)
import os
import pandas as pd

# Configuration
size = "640x640"
fov = 90
headings = [0, 90, 180, 270]
pitches = [-45, 0, 45]
include_key_in_csv = False  # avoid exposing API key in the CSV

streetview_base = "https://maps.googleapis.com/maps/api/streetview"

rows = []
for point_idx, (lat, lon) in enumerate(sampled_points):
    for heading in headings:
        for pitch in pitches:
            params = {
                "size": size,
                "location": f"{lat},{lon}",
                "heading": heading,
                "pitch": pitch,
                "fov": fov,
            }
            # URL without the key (safe to store/share)
            url_no_key = (
                f"{streetview_base}?size={params['size']}&location={params['location']}"
                f"&heading={params['heading']}&pitch={params['pitch']}&fov={params['fov']}"
            )
            # Template with placeholder for key (for agents that can inject it)
            url_with_placeholder = url_no_key + "&key={API_KEY}"
            # Optionally materialize with your key (not recommended to persist)
            url_with_key = (
                url_no_key + f"&key={API_KEY_GMP}" if include_key_in_csv else None
            )

            rows.append({
                "point_index": point_idx,
                "lat": lat,
                "lon": lon,
                "heading": heading,
                "pitch": pitch,
                "fov": fov,
                "size": size,
                "url_template": url_with_placeholder,
                "url_no_key": url_no_key,
                "url_with_key": url_with_key,
            })

sv_df = pd.DataFrame(rows)
print(f"Generated {len(sv_df)} Street View entries for {len(sampled_points)} points.")
sv_df.head()


Generated 60 Street View entries for 5 points.


,point_index,lat,lon,heading,pitch,fov,size,url_template,url_no_key,url_with_key
0,0,44.490777,11.334075,0,-45,90,640x640,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,None
1,0,44.490777,11.334075,0,0,90,640x640,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,None
2,0,44.490777,11.334075,0,45,90,640x640,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,None
3,0,44.490777,11.334075,90,-45,90,640x640,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,None
4,0,44.490777,11.334075,90,0,90,640x640,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,None


In [40]:
# Save to CSV
out_csv = "streetview_samples.csv"
sv_df.to_csv(out_csv, index=False)
print(f"Saved {len(sv_df)} rows to {out_csv}")


Saved 60 rows to streetview_samples.csv


In [41]:
# Add layer column (ground/horizon/sky) and point_id, then save layered CSV

def classify_layer(pitch: float) -> str:
    if pitch <= -30:
        return "ground"
    if pitch >= 30:
        return "sky"
    return "horizon"

sv_df["layer"] = sv_df["pitch"].apply(classify_layer)
sv_df["point_id"] = sv_df["point_index"]

# Reorder columns for readability
cols = [
    "point_id", "point_index", "lat", "lon", "layer",
    "heading", "pitch", "fov", "size",
    "url_no_key", "url_template", "url_with_key",
]
sv_df = sv_df[cols]

layered_csv = "streetview_samples_layered.csv"
sv_df.to_csv(layered_csv, index=False)
print(f"Saved layered CSV with {len(sv_df)} rows to {layered_csv}")
sv_df.head(10)


Saved layered CSV with 60 rows to streetview_samples_layered.csv


,point_id,point_index,lat,lon,layer,heading,pitch,fov,size,url_no_key,url_template,url_with_key
0,0,0,44.490777,11.334075,ground,0,-45,90,640x640,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,None
1,0,0,44.490777,11.334075,horizon,0,0,90,640x640,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,None
2,0,0,44.490777,11.334075,sky,0,45,90,640x640,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,None
3,0,0,44.490777,11.334075,ground,90,-45,90,640x640,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,None
4,0,0,44.490777,11.334075,horizon,90,0,90,640x640,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,None
5,0,0,44.490777,11.334075,sky,90,45,90,640x640,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,None
6,0,0,44.490777,11.334075,ground,180,-45,90,640x640,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,None
7,0,0,44.490777,11.334075,horizon,180,0,90,640x640,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,None
8,0,0,44.490777,11.334075,sky,180,45,90,640x640,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,None
9,0,0,44.490777,11.334075,ground,270,-45,90,640x640,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,None
